#Belajar PySpark - ArrayType pada Skema DataFrame

Dalam PySpark, StructType adalah tipe data yang digunakan untuk mendefinisikan skema (schema) dari suatu DataFrame. Data yang disimpan dalam skema dapat berupa data sederhana, seperti integer atau string, maupun data kompleks, seperti map atau array.
Dalam artikel ini, kita akan membahas salah satu tipe data kompleks yang dapat disimpan dalam Struct Type, yaitu array.


In [1]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=dd5f0e10963c4854124c6462894260b900afcd9499e7352c121b60c0e5df01cc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, ArrayType, MapType, StringType, IntegerType
from pyspark.sql.functions import array_contains, explode, split, array

In [4]:
spark = SparkSession.builder.appName("Belajar PySpark - ArrayType").getOrCreate()

##ArrayType pada Skema DataFrame

In [7]:
mySchema = StructType([
    StructField("nama", StringType(), True),
    StructField("jurusan", StringType(), True),
    StructField("nilai", ArrayType(elementType=IntegerType()), True)
])

###Menggunakan arrayType pada DataFrame

In [13]:
data = [['Agus Supono','F',[100,150,150]],
        ['Budi Sumardi','B',[200,100,150]],
        ['Dina Mariana','F',[150,150,130]],
        ['Dedi Setiadi','B', [50,100,100]]]

df = spark.createDataFrame(data, mySchema)
df.show()
df.printSchema()

+------------+-------+---------------+
|        nama|jurusan|          nilai|
+------------+-------+---------------+
| Agus Supono|      F|[100, 150, 150]|
|Budi Sumardi|      B|[200, 100, 150]|
|Dina Mariana|      F|[150, 150, 130]|
|Dedi Setiadi|      B| [50, 100, 100]|
+------------+-------+---------------+

root
 |-- nama: string (nullable = true)
 |-- jurusan: string (nullable = true)
 |-- nilai: array (nullable = true)
 |    |-- element: integer (containsNull = true)



##Mengakses ArrayType Kolom

In [14]:
df.select("nama", "nilai").show()

+------------+---------------+
|        nama|          nilai|
+------------+---------------+
| Agus Supono|[100, 150, 150]|
|Budi Sumardi|[200, 100, 150]|
|Dina Mariana|[150, 150, 130]|
|Dedi Setiadi| [50, 100, 100]|
+------------+---------------+



###Fungsi-fungsi ArrayType

####Fungsi `array_contains()`

Fungsi ini digunakan untuk memeriksa apakah suatu nilai tertentu terdapat dalam array. Fungsi ini menerima argumen berupa kolom yang bertipe array, dan mengembalikan nilai berupa kolom bertipe boolean.

Misalnya pada contoh di atas, kita dapat memeriksa apakah kolom “nilai” yang bertipe array mengandung nilai 200. Hasilnya akan kita simpan ke dalam kolom bernama `mengandung_nilai_200`


In [24]:
df.select(df.nama,
          array_contains(df.nilai,200)
          .alias("mengandung_nilai_200")).show()

+------------+--------------------+
|        nama|mengandung_nilai_200|
+------------+--------------------+
| Agus Supono|               false|
|Budi Sumardi|                true|
|Dina Mariana|               false|
|Dedi Setiadi|               false|
+------------+--------------------+



####Fungsi `Explode()`

Fungsi explode digunakan untuk mengubah setiap elemen array menjadi satu baris atau record tersendiri. Misalnya untuk contoh di atas :


In [25]:
df.select("nama", "jurusan", explode(df["nilai"])).show()

+------------+-------+---+
|        nama|jurusan|col|
+------------+-------+---+
| Agus Supono|      F|100|
| Agus Supono|      F|150|
| Agus Supono|      F|150|
|Budi Sumardi|      B|200|
|Budi Sumardi|      B|100|
|Budi Sumardi|      B|150|
|Dina Mariana|      F|150|
|Dina Mariana|      F|150|
|Dina Mariana|      F|130|
|Dedi Setiadi|      B| 50|
|Dedi Setiadi|      B|100|
|Dedi Setiadi|      B|100|
+------------+-------+---+



####Fungsi `Split()`

Fungsi ini digunakan untuk membentuk array dari sebuah string. String dipecah berdasarkan karakter pemisah/delimiter tertentu, dan bagian-bagiannya dibentuk menjadi elemen dari sebuah array.

Misalnya dari contoh di atas, kita akan membuat kolom `arrayNama` dari kolom `nama`. Kita gunakan spasi sebagai delimiter.


In [26]:
df.select("nama", "jurusan", "nilai",
          split(df.nama," ").alias("arrayNama")).show()

+------------+-------+---------------+---------------+
|        nama|jurusan|          nilai|      arrayNama|
+------------+-------+---------------+---------------+
| Agus Supono|      F|[100, 150, 150]| [Agus, Supono]|
|Budi Sumardi|      B|[200, 100, 150]|[Budi, Sumardi]|
|Dina Mariana|      F|[150, 150, 130]|[Dina, Mariana]|
|Dedi Setiadi|      B| [50, 100, 100]|[Dedi, Setiadi]|
+------------+-------+---------------+---------------+



####Fungsi `array()`

Fungsi ini digunakan untuk membentuk kolom bertipe array dari gabungan kolom-kolom lain. Misalnya kita memiliki dataframe seperti berikut ini


In [21]:
data1 = [['Agus Supono','F',100,150,150],
        ['Budi Sumardi','B',200,100,150],
        ['Dina Mariana','F',150,150,130],
        ['Dedi Setiadi','B', 50,100,100]]

mycolumns = ["nama","jurusan","nilai1","nilai2","nilai3"]
df1 = spark.createDataFrame(data1, mycolumns)
df1.show()
df1.printSchema()

+------------+-------+------+------+------+
|        nama|jurusan|nilai1|nilai2|nilai3|
+------------+-------+------+------+------+
| Agus Supono|      F|   100|   150|   150|
|Budi Sumardi|      B|   200|   100|   150|
|Dina Mariana|      F|   150|   150|   130|
|Dedi Setiadi|      B|    50|   100|   100|
+------------+-------+------+------+------+

root
 |-- nama: string (nullable = true)
 |-- jurusan: string (nullable = true)
 |-- nilai1: long (nullable = true)
 |-- nilai2: long (nullable = true)
 |-- nilai3: long (nullable = true)



Kita dapat membentuk kolom `nilai` yang dibentuk dari kolom-kolom `nilai1`, `nilai2`, dan `nilai3`


In [58]:
df1.select(df1.nama, df1.jurusan,
           array(df1.nilai1,df1.nilai2,df1.nilai2)
           .alias("nilai")).show()

+------------+-------+---------------+
|        nama|jurusan|          nilai|
+------------+-------+---------------+
| Agus Supono|      F|[100, 150, 150]|
|Budi Sumardi|      B|[200, 100, 100]|
|Dina Mariana|      F|[150, 150, 150]|
|Dedi Setiadi|      B| [50, 100, 100]|
+------------+-------+---------------+

